<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
تشخیص ناهنجاری
</font>
</h1>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>

</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در این قسمت کتاب‌خانه‌ها و ابزار مورد نیاز خود را 
<code>import</code>
 کنید 
 و
  فایل داده‌ها را که در پوشه‌ی 
  <code>Data</code>
ذخیره‌شده‌اند را بخوانید و وارد محیط کار خود کنید.
</font>
</p>

In [13]:
# Load the data and prepare it for training the model
import pandas as pd
import numpy as np
import os
import glob

data_path = './data/'

# Reading all csv files from the directory 
all_files = glob.glob(os.path.join(data_path, "*_test.csv"))

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
        در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی دلخواهتان، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.
    در این قسمت شما می‌توانید بخشی از داده‌ی موجود را برای اعتبارسنجی در نظر بگیرید.
</font>
</p>

In [14]:
# Creating a dict for each sensor
dataframes = {}
for file in all_files:
    # Extracting the sensor's name from the file
    sensor_name = os.path.basename(file).replace('_test.csv', '')
    df = pd.read_csv(file)

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    df.columns = [sensor_name]
    dataframes[sensor_name] = df

print(f"{len(dataframes)} sensors have been loaded.")

# Finding the sensor with the highest rate
highest_rate_sensor = max(dataframes, key=lambda k: len(dataframes[k]))
print(f"The highest rate sensor: {highest_rate_sensor}")

# Getting the time index for the sensor with the highest rate
highest_rate_index = dataframes[highest_rate_sensor].index

# Creating a new df
merged_df = pd.DataFrame(index=highest_rate_index)

# Merging the dfs
for sensor_name, df in dataframes.items():
    resampled_df = df.reindex(highest_rate_index).interpolate(method='time')
    resampled_df = resampled_df.ffill().bfill()
    merged_df[sensor_name] = resampled_df

print("\nMerged df info:")
merged_df.info()
print("\nHead of the merged df info:")
print(merged_df.head())

15 sensors have been loaded.
The highest rate sensor: BearingTemp_C

Merged df info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 45900 entries, 2025-01-02 05:45:00 to 2025-01-02 18:29:59
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   BearingTemp_C     45900 non-null  float64
 1   Current_A         45900 non-null  float64
 2   FlowRate_L_min    45900 non-null  float64
 3   Humidity_pct      45900 non-null  float64
 4   OilLevel_cm       45900 non-null  float64
 5   pH_units          45900 non-null  float64
 6   Power_kW          45900 non-null  float64
 7   Pressure_kPa      45900 non-null  float64
 8   Speed_RPM         45900 non-null  float64
 9   Temperature_C     45900 non-null  float64
 10  Torque_Nm         45900 non-null  float64
 11  VibAccel_m_s2     45900 non-null  float64
 12  VibDisp_mm        45900 non-null  float64
 13  VibVelocity_mm_s  45900 non-null  float64
 14  Voltage_V     

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>



In [15]:
# Model development and training
from sklearn.ensemble import IsolationForest

model = IsolationForest(
    n_estimators=200,
    contamination=0.02,
    max_features=1.0, # Use all features for each tree
    random_state=42,
    n_jobs=-1 # Use all available CPU cores
)

print("Training the Isolation Forest ...")
model.fit(merged_df)
print("Trained.")

Training the Isolation Forest ...
Trained.


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>



In [16]:
# Predict and create submission file
predictions = model.predict(merged_df)

# We should change -1 with 0 because of the prediction format
submission_predictions = np.where(predictions == -1, 1, 0)

submission_df = pd.DataFrame({'prediction': submission_predictions})

submission_df.to_csv('submission.csv', index=False)

print("Submission created successfully.")
print(f"Prediction: {submission_df['prediction'].sum()}")

Submission created successfully.
Prediction: 918


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>


In [17]:
from sklearn.metrics import f1_score
# print(f"F1 score: {f1_score(submission_df['prediction'], )}")

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [18]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'notebook.ipynb')):
    %notebook -e notebook.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission_df.to_csv('submission.csv', index=False)
file_names = ['notebook.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['notebook.ipynb', 'submission.csv']
